# Dynamo-based perturbation analysis

## Library imports

In [1]:
import numpy as np
import pandas as pd

import dynamo as dyn
import scanpy as sc
from dynamo.preprocessing import Preprocessor

from rgv_tools import DATA_DIR
from rgv_tools.perturbation import (
    delta_to_probability,
    density_likelihood_dyn,
    get_list_name,
    Multiple_TFScanning_perturbation_dyn,
    split_elements,
    TFScanning_perturbation_dyn,
)

/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packag

## Constants

In [2]:
DATASET = "zebrafish"

In [3]:
SAVE_DATA = True
if SAVE_DATA:
    (DATA_DIR / DATASET / "results").mkdir(parents=True, exist_ok=True)

In [4]:
genes = ["nr2f5", "sox9b", "twist1b", "ets1"]

TERMINAL_STATES_KO = [
    "mNC_head_mesenchymal",
    "mNC_arch2",
    "mNC_hox34",
    "Pigment_gch2",
]

TERMINAL_STATES_PERTURB = [
    "mNC_head_mesenchymal",
    "mNC_arch2",
    "mNC_hox34",
    "Pigment",
]

In [5]:
single_ko = ["rarga", "rxraa", "nr2f5", "fli1a", "tfec", "elk3", "mitfa", "ets1", "nr2f2", "elf1", "ebf3a"]
multiple_ko = ["fli1a_elk3", "tfec_mitfa_bhlhe40", "mitfa_tfec", "mitfa_tfec_tfeb"]

## Data loading

In [6]:
adata = sc.read_h5ad(DATA_DIR / DATASET / "processed" / "adata_preprocessed_cr.h5ad")

In [7]:
adata.X = adata.layers["matrix"].copy()

## Processing by dynamo

In [8]:
preprocessor = Preprocessor()
preprocessor.preprocess_adata(adata, recipe="monocle")

|-----> Running monocle preprocessing pipeline...
|-----------> filtered out 0 outlier cells
|-----------> filtered out 21 outlier genes
|-----> PCA dimension reduction
|-----> <insert> X_pca to obsm in AnnData Object.
|-----> [Preprocessor-monocle] completed [1.6919s]


In [9]:
dyn.tl.dynamics(adata)

|-----> dynamics_del_2nd_moments_key is None. Using default value from DynamoAdataConfig: dynamics_del_2nd_moments_key=False
|-----------> removing existing M layers:[]...
|-----------> making adata smooth...
|-----> calculating first/second moments...
|-----? layer X_velocity is not in any of the (['X_spliced', 'X_unspliced'], ['X_new', 'X_total'], ['X_uu', 'X_ul', 'X_su', 'X_sl']) groups, skipping...
|-----? layer X_latent_time_velovi is not in any of the (['X_spliced', 'X_unspliced'], ['X_new', 'X_total'], ['X_uu', 'X_ul', 'X_su', 'X_sl']) groups, skipping...
|-----? layer X_fit_t is not in any of the (['X_spliced', 'X_unspliced'], ['X_new', 'X_total'], ['X_uu', 'X_ul', 'X_su', 'X_sl']) groups, skipping...
|-----? layer X_Mu is not in any of the (['X_spliced', 'X_unspliced'], ['X_new', 'X_total'], ['X_uu', 'X_ul', 'X_su', 'X_sl']) groups, skipping...
|-----? layer X_Ms is not in any of the (['X_spliced', 'X_unspliced'], ['X_new', 'X_total'], ['X_uu', 'X_ul', 'X_su', 'X_sl']) groups,

estimating gamma: 100%|██████████| 988/988 [00:04<00:00, 228.75it/s]


AnnData object with n_obs × n_vars = 697 × 988
    obs: 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts', 'cell_type', 'stage', 'cell_type2', 'macrostates', 'cell_type_old', 'macrostates_fwd', 'term_states_fwd', 'term_states_fwd_probs', 'visits', 'nGenes', 'nCounts', 'pMito', 'pass_basic_filter', 'Size_Factor', 'initial_cell_size', 'spliced_Size_Factor', 'initial_spliced_cell_size', 'unspliced_Size_Factor', 'initial_unspliced_cell_size', 'Ms_Size_Factor', 'initial_Ms_cell_size', 'latent_time_velovi_Size_Factor', 'initial_latent_time_velovi_cell_size', 'fit_t_Size_Factor', 'initial_fit_t_cell_size', 'Mu_Size_Factor', 'initial_Mu_cell_size', 'velocity_Size_Factor', 'initial_velocity_cell_size', 'ntr'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'gene_count_corr', 'is_tf', 'TF', 'velocity_genes', 'fit_beta', 'fit_gamma', 'fit_scaling', 'nCells', 'nCounts', 'pass_basic_filter', 'frac', 'use_for_pca', 'ntr', 'beta', 'gamma', 'half_life', 'alpha_b

In [10]:
dyn.tl.reduceDimension(adata)

|-----> retrieve data for non-linear dimension reduction...
|-----? adata already have basis umap. dimension reduction umap will be skipped! 
set enforce=True to re-performing dimension reduction.
|-----> [UMAP] completed [0.0102s]


In [11]:
dyn.tl.cell_velocities(adata, basis="pca")

|-----> [calculating transition matrix via pearson kernel with sqrt transform.] in progress: 100.0000%|-----> [calculating transition matrix via pearson kernel with sqrt transform.] completed [0.5041s]
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [0.1790s]


AnnData object with n_obs × n_vars = 697 × 988
    obs: 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts', 'cell_type', 'stage', 'cell_type2', 'macrostates', 'cell_type_old', 'macrostates_fwd', 'term_states_fwd', 'term_states_fwd_probs', 'visits', 'nGenes', 'nCounts', 'pMito', 'pass_basic_filter', 'Size_Factor', 'initial_cell_size', 'spliced_Size_Factor', 'initial_spliced_cell_size', 'unspliced_Size_Factor', 'initial_unspliced_cell_size', 'Ms_Size_Factor', 'initial_Ms_cell_size', 'latent_time_velovi_Size_Factor', 'initial_latent_time_velovi_cell_size', 'fit_t_Size_Factor', 'initial_fit_t_cell_size', 'Mu_Size_Factor', 'initial_Mu_cell_size', 'velocity_Size_Factor', 'initial_velocity_cell_size', 'ntr'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'gene_count_corr', 'is_tf', 'TF', 'velocity_genes', 'fit_beta', 'fit_gamma', 'fit_scaling', 'nCells', 'nCounts', 'pass_basic_filter', 'frac', 'use_for_pca', 'ntr', 'beta', 'gamma', 'half_life', 'alpha_b

In [12]:
dyn.vf.VectorField(adata, basis="pca")

|-----> VectorField reconstruction begins...
|-----> Retrieve X and V based on basis: PCA. 
        Vector field will be learned in the PCA space.
|-----> Learning vector field with method: sparsevfc.
|-----> [SparseVFC] begins...
|-----> Sampling control points based on data velocity magnitude...
|-----> [SparseVFC] completed [0.4913s]
|-----> [VectorField] completed [0.5661s]


In [13]:
adata_perturb = adata.copy()

## Perturbation prediction

Function based perturbation

In [14]:
single_ko = set(single_ko).intersection(adata.var_names)
single_ko = list(single_ko)

In [15]:
start_indices = np.where(adata.obs["cell_type"].isin(["NPB_nohox"]))[0]

In [16]:
cand_list = single_ko + multiple_ko

In [17]:
dl_score_all = []
dl_sig_all = []
for TF in cand_list:
    TF_list = split_elements([TF])[0]
    dl_score, dl_sig, _, _ = density_likelihood_dyn(
        adata, TF_list, start_indices, TERMINAL_STATES_KO, n_simulations=1000
    )
    dl_score_all.append(dl_score)
    dl_sig_all.append(dl_sig)

  0%|          | 0/697 [00:00<?, ?cell/s]

  0%|          | 0/697 [00:00<?, ?cell/s]

|-----> In silico knockout ['elk3']
|-----> Project the high dimensional vector field after KO to umap.
|-----> [calculating transition matrix via pearson kernel with sqrt transform.] in progress: 100.0000%|-----> [calculating transition matrix via pearson kernel with sqrt transform.] completed [0.1830s]
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [0.1614s]


  0%|          | 0/697 [00:00<?, ?cell/s]

  0%|          | 0/697 [00:00<?, ?cell/s]

...

  0%|          | 0/697 [00:00<?, ?cell/s]

  0%|          | 0/697 [00:00<?, ?cell/s]

|-----> In silico knockout ['mitfa', 'tfec', 'tfeb']
|-----> Project the high dimensional vector field after KO to umap.
|-----> [calculating transition matrix via pearson kernel with sqrt transform.] in progress: 100.0000%|-----> [calculating transition matrix via pearson kernel with sqrt transform.] completed [0.1791s]
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [0.1591s]


  0%|          | 0/697 [00:00<?, ?cell/s]

  0%|          | 0/697 [00:00<?, ?cell/s]

In [18]:
pred_m_single = pd.DataFrame(
    np.array(dl_score_all[: len(single_ko)]), index=cand_list[: len(single_ko)], columns=TERMINAL_STATES_KO
)
pred_m_multiple = pd.DataFrame(
    np.array(dl_score_all[len(single_ko) :]), index=cand_list[len(single_ko) :], columns=TERMINAL_STATES_KO
)

In [19]:
pval_m_single = pd.DataFrame(
    np.array(dl_sig_all[: len(single_ko)]), index=cand_list[: len(single_ko)], columns=TERMINAL_STATES_KO
)
pval_m_multiple = pd.DataFrame(
    np.array(dl_sig_all[len(single_ko) :]), index=cand_list[len(single_ko) :], columns=TERMINAL_STATES_KO
)

In [20]:
## Perform KO single screening using function based perturbation
coef_KO = delta_to_probability(pred_m_single, k=0.005)

## Perform KO multiple screening using function based perturbation
coef_KO_multiple = delta_to_probability(pred_m_multiple, k=0.005)

In [21]:
coef_KO = coef_KO.loc[single_ko, TERMINAL_STATES_KO]
coef_KO_multiple = coef_KO_multiple.loc[multiple_ko, TERMINAL_STATES_KO]

pval_KO = pval_m_single.loc[single_ko, TERMINAL_STATES_KO]
pval_KO_multiple = pval_m_multiple.loc[multiple_ko, TERMINAL_STATES_KO]

## Gene expression perturbation

In [22]:
## Dynamo (perturbation)
perturbation_dyn = TFScanning_perturbation_dyn(adata, 8, "cell_type", TERMINAL_STATES_KO, single_ko)

|-----> In silico perturbation of single-cells and prediction of cell fate after perturbation...
|-----> Retrive X_pca, PCs, pca_mean...
|-----> Calculate perturbation effect matrix via \delta Y = J \dot \delta X....


calculating Jacobian for each cell: 100%|██████████| 697/697 [00:00<00:00, 223828.95it/s]

|-----> project the pca perturbation vector to low dimensional space....


|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [0.1890s]
|-----> you can use dyn.pl.streamline_plot(adata, basis='pca_perturbation') to visualize the perturbation vector
Done elk3
|-----> In silico perturbation of single-cells and prediction of cell fate after perturbation...
|-----> Retrive X_pca, PCs, pca_mean...
|-----> Calculate perturbation effect matrix via \delta Y = J \dot \delta X....
|-----> project the pca perturbation vector to low dimensional space....
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [0.1608s]
|-----> you can use dyn.pl.streamline_plot(adata, basis='pca_perturbation') to visualize the perturbation vector
Done fli1a
|-----> In silico perturbation of single-cells and prediction of cell fate after perturbation...
|-----> Retrive X_

In [23]:
## Dynamo (perturbation) in multiple
multiple_ko = ["mitfa_tfec_tfeb", "fli1a_elk3", "mitfa_tfec", "tfec_mitfa_bhlhe40"]
multiple_ko_list = split_elements(multiple_ko)
perturbation_dyn_multiple = Multiple_TFScanning_perturbation_dyn(
    adata, 8, "cell_type", TERMINAL_STATES_KO, multiple_ko_list
)

|-----> In silico perturbation of single-cells and prediction of cell fate after perturbation...
|-----> Retrive X_pca, PCs, pca_mean...
|-----> Calculate perturbation effect matrix via \delta Y = J \dot \delta X....
|-----> project the pca perturbation vector to low dimensional space....
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [0.1630s]
|-----> you can use dyn.pl.streamline_plot(adata, basis='umap_perturbation') to visualize the perturbation vector
Done mitfa_tfec_tfeb
|-----> In silico perturbation of single-cells and prediction of cell fate after perturbation...
|-----> Retrive X_pca, PCs, pca_mean...
|-----> Calculate perturbation effect matrix via \delta Y = J \dot \delta X....
|-----> project the pca perturbation vector to low dimensional space....
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting vel

In [24]:
## Perform KO screening using function based perturbation
coef_perturb = pd.DataFrame(np.array(perturbation_dyn["coefficient"]))
coef_perturb.index = perturbation_dyn["TF"]
coef_perturb.columns = get_list_name(perturbation_dyn["coefficient"][0])
coef_perturb = coef_perturb.loc[single_ko, TERMINAL_STATES_PERTURB]

## Perform perturbation screening using gene expression based perturbation
coef_perturb_multiple = pd.DataFrame(np.array(perturbation_dyn_multiple["coefficient"]))
coef_perturb_multiple.index = perturbation_dyn_multiple["TF"]
coef_perturb_multiple.columns = get_list_name(perturbation_dyn_multiple["coefficient"][0])
coef_perturb_multiple = coef_perturb_multiple.loc[multiple_ko, TERMINAL_STATES_PERTURB]

In [25]:
if SAVE_DATA:
    coef_KO.to_csv(DATA_DIR / DATASET / "results" / "dynamo_KO_single.csv")
    coef_KO_multiple.to_csv(DATA_DIR / DATASET / "results" / "dynamo_KO_multiple.csv")
    pval_KO.to_csv(DATA_DIR / DATASET / "results" / "dynamo_KO_single_pval.csv")
    pval_KO_multiple.to_csv(DATA_DIR / DATASET / "results" / "dynamo_KO_multiple_pval.csv")

    coef_perturb.to_csv(DATA_DIR / DATASET / "results" / "dynamo_perturb_single.csv")
    coef_perturb_multiple.to_csv(DATA_DIR / DATASET / "results" / "dynamo_perturb_multiple.csv")